# Resumen Selección mejor modelo

## Validación cruzada

Dado un Estimador or Pipeline: *`model`*

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

# Definimos el tipo de validación cruzada
cv = ShuffleSplit(n_splits=40, test_size=0.3, random_state=0)

# Aplicamos la validación cruzada
cv_results = cross_validate( model, data, target, cv=cv, scoring="neg_mean_absolute_error", return_estimator=True)

# transformamos los resultados en un DataFrame para procesarlo mejor
cv_results_df = pd.DataFrame(cv_results)


In [ ]:
# obtener los errores de entrenamiento y test invirtiendo los negativos
scores[["train error", "test error"]] = -cv_results[["train_score", "test_score"]]

In [ ]:
# graficamos
import matplotlib.pyplot as plt

scores.plot.hist(bins=50, edgecolor="black")
plt.xlabel("Error absoluto medio (k$)")
_ = plt.title("Distribución de errores de entrenamiento y pruebas en la validación cruzada")

## Curva de validación

**Iteramos sobre un hiperparámetro para ver cómo influye en las mátricas**

- Tener en cuenta que el nombre del hiperparámetro es:
    - Si el modelo es un estimador, el nombre tal cual. e.g. `max_depth`
    - Si es un pipeline, `<nombre_del_step>__<nombre_hiperparámentro>`. e,g, `regressor__max_depth`

- `cv` puede ser un número (K-Fold) o un objeto (e.g. Shuffle-Split)

In [ ]:
from sklearn.model_selection import validation_curve

max_depth = [1, 5, 10, 15, 20, 25]

train_scores, test_scores = validation_curve(
    model, data, target, param_name="regressor__max_depth", param_range=max_depth,
    cv=cv, scoring="neg_mean_absolute_error", n_jobs=2)

train_errors, test_errors = -train_scores, -test_scores

### Graficamos los errores de entrenamiento y prueba (así como sus desviaciones).

Buscamos el menor error de prueba, con el menor error de entrenamiento, pero sin sobreajustar (mayor que 0)


In [ ]:
# Graficamos los errores de entrenamiento y prueba

plt.plot(max_depth, train_errors.mean(axis=1), label="Error de entrenamiento")
plt.plot(max_depth, test_errors.mean(axis=1), label="Error de prueba")
plt.legend()

plt.xlabel("Profundidad máxima del árbol de decisión (max_depth)")
plt.ylabel("Error absoluto medio (k$)")
plt.title("Curva de validación para el árbol de decisión")

In [ ]:
# Misma gráfica,añadiendo información para mostrar la desviación estándar de los errores.

plt.errorbar(max_depth, train_errors.mean(axis=1),
             yerr=train_errors.std(axis=1), label='Error de entrenamiento')
plt.errorbar(max_depth, test_errors.mean(axis=1),
             yerr=test_errors.std(axis=1), label='Error de prueba')
plt.legend()

plt.xlabel("Profundidad máxima del árbol de decisión (max_depth)")
plt.ylabel("Error absoluto medio (k$)")
plt.title("Curva de validación para el árbol de decisión")

## Curva de aprendizaje

**Iteramos sobre el volumen de datos para ver cómo influye en las métricas y si converge**

In [ ]:
import numpy as np
from sklearn.model_selection import learning_curve

train_sizes = np.linspace(0.1, 1.0, num=5, endpoint=True)


results = learning_curve(model, data, target, train_sizes=train_sizes, cv=cv, scoring='neg_mean_absolute_error', n_jobs=2)

train_size, train_scores, test_scores = results[:3]

train_errors, test_errors = -train_scores, -test_scores

### Graficamos los errores de entrenamiento y prueba (así como sus desviaciones).

- Buscamos el menor error de prueba, con el menor error de entrenamiento, pero sin sobreajustar (mayor que 0).
- Verificamos si se resducen o mantienen conforme incrementamos las muestars (convergen)


In [ ]:
# graficar la curva.

import matplotlib.pyplot as plt

plt.errorbar(train_size, train_errors.mean(axis=1), yerr=train_errors.std(axis=1), label="Error de entrenamiento")
plt.errorbar(train_size, test_errors.mean(axis=1), yerr=test_errors.std(axis=1), label="Error de prueba")
plt.legend()

plt.xscale("log")
plt.xlabel("Número de muestras en el conjunto de entrenamiento")
plt.ylabel("Error absoluto medio (k $)")
plt.title("Curva de aprendizaje para el árbol de decisión")